In [7]:
# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

In [8]:
def detect_and_predict_mask(frame, faceNet, maskNet):
    # prendre les dimmensions de l'image et construire un blob avec
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),(104.0, 177.0, 123.0))
    # passer le blob à notre reseau de Neurone pour faire la detection sur le face
    faceNet.setInput(blob)
    # moove one
    detections = faceNet.forward()
    print(detections.shape)
    #initialisation des lists: faces, leurs locations, et la liste des predictions
    faces = []
    locs = []
    pred = []
    
    for i in range(0 ,detections.shape[2]):
        # extraire la confidence de probabilité associaée à la detection
        confidence = detections[0, 0, i, 2]
        # filtrer pour prendre uniquement ceux > 0.5
        if confidence > 0.5:
            # compute the (x, y)-coordinates of the bounding box for the object
            box = detections[0,0,i, 3:7]*np.array([w,h,w,h])
            (startX,startY, endX,endY) = box.astype('int')
            # ensure the bounding boxes fall within the dimensions of the frame
            (startX,startY) = (max(0,startX), max(0,startY))
            (endX,endY) = (min(w-1,endX), min(h-1,endY))
            #extraire la face ROI et le convertir from BGR to RG, resizer puit preprocess it
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224 , 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            # lists
            faces.append(face)
            locs.append((startX,startY, endX,endY))
    
    # si il ya une image dont la taille >0 alors on fait la prediction
    if len(faces) >0:
        #convertion en tableau numpy
        faces = np.array(faces,dtype="float32")
        #prediction
        pred = maskNet.predict(faces, batch_size=32) #pour des soucis de rapiditer on applique batch size sur each prediction
        
    return (locs,pred)# return un tuple

In [9]:
# chargement de serialized face detector model from disk
prototxtPath = "deploy.prototxt"
weightsPath = "res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

In [12]:
#Chargement du model
#maskNet = load_model("last_Mask_Model.h5")
maskNet = load_model("mask_detector.Model")
#Video
#video = VideoStream(src=0).stop()  
video = VideoStream(src=0).start()

In [11]:
while True:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 400 pixel
    frame = video.read()
    frame = imutils.resize(frame,width=400)
    #prediction
    (locs,pred) = detect_and_predict_mask(frame, faceNet, maskNet)
    #parcourir les faces detectés
    for (box,pred) in zip(locs,pred):
        # decompresser les lignes box rectangle et faire la predictions
        (startX,startY, endX,endY) = box
        (Mask, withoutMasK) = pred
        #donner un nom et une couleur à notre box
        label = "Mask" if Mask > withoutMasK else "No Mask"
        color = (0,255,0) if label == "Mask" else (0,0,255)
        # include la probability dans le label
        label = "{}: {:.2f}%".format(label, max(Mask, withoutMasK) * 100)
        #Dessin du rectangle(img, top_left,bottom_right,color,epaisseur)
        cv2.rectangle(frame, (startX,startY),(endX,endY), color,2)
        #Text
        cv2.putText(frame, label, (startX, startY - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        
    # show the output frame
    cv2.imshow("Mask_Detection",frame)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# do a bit of cleanup
cv2.destroyAllWindows()
video.stop()

(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200

(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200